In [6]:
#P.279 LSTM的權重參數因多了三個gate，所以是RNN的四倍
#RNN的權重(300+1+50) x 50 = 17550
#LSTM的權重17550 x 4(三個gate其中一個有兩個) = 70200
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, LSTM 

num_neurons = 50 
model = Sequential() 
model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
#(300+1+50)*50*4=70200
model.add(Dropout(.2))  #shape(400,50)
model.add(Flatten()) #shape(0,20000)
model.add(Dense(1, activation='sigmoid')) #20000+1=20001
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy']) 
model.summary() 

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 400, 50)           70200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 20001     
Total params: 90,201
Trainable params: 90,201
Non-trainable params: 0
_________________________________________________________________


In [1]:
#P.231 IMDB電影評論dataset的資料前處理
#為文檔標記適當label (1(正評) or 0(負評))
#混洗(shuffled)所有樣本，使樣本抽出時不會全部正評或負評
import glob
import os

from random import shuffle

def pre_process_data(filepath): 
    """ ... This is dependent on your training data source but we will
    try to generalize it as best as possible. ... """ 
    positive_path = os.path.join(filepath, 'pos')#'Documents/Python/NLP/aclImdb/train/pos'
    negative_path = os.path.join(filepath, 'neg')
    pos_label = 1 
    neg_label = 0 
    dataset = [] 
    
    for filename in glob.glob(os.path.join(positive_path, '*.txt')): 
        #*表是該路徑下每個.txt檔案的名稱都跑一次
        #glob.glob()返回所有匹配的文件路徑列表
        with open(filename, 'r', encoding="utf-8") as f: 
            dataset.append((pos_label, f.read())) 
            
    for filename in glob.glob(os.path.join(negative_path, '*.txt')): 
        with open(filename, 'r', encoding="utf-8") as f: 
            dataset.append((neg_label, f.read())) 
            
    shuffle(dataset) 
    
    return dataset

dataset = pre_process_data('./aclimdb/train') 
dataset

In [2]:
#P.232 對每個文檔裡的每個word做分解
#並合併每個word的word_vectors為sample_vecs(文檔的vector)，此時為每個文檔裡的所有word_vectors
#再把sample_vecs合併成vectorized_data，此時包含所有文檔的每個word的vector(頻率含意)
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
#from nlpia.loaders import get_data 
#word_vectors = get_data('w2v', limit=200000)
from gensim.models import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True , limit=200000)

def tokenize_and_vectorize(dataset): 
    tokenizer = TreebankWordTokenizer() 
    vectorized_data = [] 
    expected = [] 
    for sample in dataset: 
        tokens = tokenizer.tokenize(sample[1])#抓出sample的文字部分做分解
        sample_vecs = [] 
        for token in tokens: 
            try: 
                sample_vecs.append(word_vectors[token])
                #word_vectors[token]每個字在word_vectors的vector(頻率含意) 
                
            except KeyError: 
                pass # No matching token in the Google w2v vocab 
            
        vectorized_data.append(sample_vecs) 
        
    return vectorized_data

In [3]:
#P.233 抓出sample的label部分 
def collect_expected(dataset):
    """ Peel off the target values from the dataset """ 
    expected = [] 
    for sample in dataset: 
        expected.append(sample[0])#label的部分 
    return expected

In [4]:
#P.234-235 padding(填充)不夠的data或truncate(截斷)多出的data
def pad_trunc(data, maxlen): 
    """ For a given dataset pad with zero vectors or truncate to maxlen """ 
    new_data = []
    # Create a vector of 0s the length of our word vectors 
    zero_vector = [] 
    for _ in range(len(data[0][0])):#每個文檔裡每個word的word_vectors(分數含意)長度都是300  
        zero_vector.append(0.0)#生成一個長度為300的zero vectors
        
    for sample in data:#每個文檔裡的字 
        if len(sample) > maxlen:#字的長度大於maxlen=400
            temp = sample[:maxlen] #只擷取到maxlen
        elif len(sample) < maxlen:
            temp = sample 
            # Append the appropriate number 0 vectors to the list 
            additional_elems = maxlen - len(sample) #不足400的長度
            for _ in range(additional_elems): 
                temp.append(zero_vector) #把不足的字的長度以zero vectors加上
        else: 
            temp = sample 
        new_data.append(temp) 
    return new_data

In [5]:
#P.285 資料前處理
import numpy as np

dataset = pre_process_data('./aclimdb/train') 
vectorized_data = tokenize_and_vectorize(dataset) 
expected = collect_expected(dataset) 

split_point = int(len(vectorized_data)*.4)
split_point_end = int(len(vectorized_data)*.5)

x_train = vectorized_data[:split_point] 
y_train = expected[:split_point] 
x_test = vectorized_data[split_point:split_point_end] 
y_test = expected[split_point:split_point_end]

maxlen = 400 
batch_size = 32 
embedding_dims = 300 
epochs = 2

x_train = pad_trunc(x_train, maxlen) 
x_test = pad_trunc(x_test, maxlen) 
x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims)) 
y_train = np.array(y_train) 
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims)) 
y_test = np.array(y_test)

In [6]:
#P.286 建模
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, LSTM 

num_neurons = 50 
model = Sequential() 
model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
#(300+1+50)*50*4=70200
model.add(Dropout(.2))  #shape(400,50)
model.add(Flatten()) #shape(0,20000)
model.add(Dense(1, activation='sigmoid')) #20000+1=20001
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy']) 
model.summary() 

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 400, 50)           70200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 20001     
Total params: 90,201
Trainable params: 90,201
Non-trainable params: 0
_________________________________________________________________


In [7]:
#P.286 fit訓練模型
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test)) 

Train on 10000 samples, validate on 2500 samples
Epoch 1/2
10000/10000 [==============================] - 257s 26ms/step - loss: 0.5268 - accuracy: 0.7364 - val_loss: 0.5267 - val_accuracy: 0.7388
Epoch 2/2
10000/10000 [==============================] - 248s 25ms/step - loss: 0.3933 - accuracy: 0.8245 - val_loss: 0.3723 - val_accuracy: 0.8368


In [8]:
#P.286 save model
model_structure = model.to_json() 
with open("lstm_model1.json", "w") as json_file: 
    json_file.write(model_structure)
model.save_weights("lstm_weights1.h5")

In [9]:
#P.287
from keras.models import model_from_json 
with open("lstm_model1.json", "r") as json_file: 
    json_string = json_file.read() 
model = model_from_json(json_string)
model.load_weights('lstm_weights1.h5')

In [10]:
#P.287
sample_1 = """I hate that the dismal weather had me down for so long,\
when will it break! Ugh, when does happiness return? The sun is\
blinding and the puffy clouds are too thin. I can't wait for the weekend."""

vec_list = tokenize_and_vectorize([(1, sample_1)])

test_vec_list = pad_trunc(vec_list, maxlen)

test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

print("Sample's sentiment, 1 - pos, 2 - neg : {}".format(model.predict_classes(test_vec))) 

print("Raw output of sigmoid function: {}".format(model.predict(test_vec))) 

Sample's sentiment, 1 - pos, 2 - neg : [[0]]
Raw output of sigmoid function: [[0.25997096]]


In [11]:
#P.289 發現token長度設得太高，可調降長度至平均文檔token長度(Avg length)
def test_len(data, maxlen): 
    total_len = truncated = exact = padded = 0 
    for sample in data: 
        total_len += len(sample) 
        if len(sample) > maxlen: 
            truncated += 1 
        elif len(sample) < maxlen: 
            padded += 1 
        else: 
            exact +=1 
    print('Padded: {}'.format(padded)) 
    print('Equal: {}'.format(exact)) 
    print('Truncated: {}'.format(truncated)) 
    print('Avg length: {}'.format(total_len/len(data)))

dataset = pre_process_data('./aclimdb/train') 
vectorized_data = tokenize_and_vectorize(dataset) 
test_len(vectorized_data, 400)

Padded: 22560
Equal: 12
Truncated: 2428
Avg length: 202.43204


In [12]:
#P.289-290 調降長度至平均文檔token長度
import numpy as np 
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, LSTM 

maxlen = 200 
batch_size = 32 
embedding_dims = 300 
epochs = 2 
num_neurons = 50 
dataset = pre_process_data('./aclimdb/train') 
vectorized_data = tokenize_and_vectorize(dataset) 
expected = collect_expected(dataset) 

split_point = int(len(vectorized_data)*.4)
split_point_end = int(len(vectorized_data)*.5)
x_train = vectorized_data[:split_point] 
y_train = expected[:split_point] 
x_test = vectorized_data[split_point:split_point_end] 
y_test = expected[split_point:split_point_end]

x_train = pad_trunc(x_train, maxlen) 
x_test = pad_trunc(x_test, maxlen)
x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims)) 
y_train = np.array(y_train) 
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims)) 
y_test = np.array(y_test)

In [13]:
#P.290
model = Sequential() 
model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims))) 
#(300+1+50)*50*4=70200
model.add(Dropout(.2))  #shape(200,50)
model.add(Flatten()) #shape(0,10000)
model.add(Dense(1, activation='sigmoid')) #10000+1=10001
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy']) 
model.summary() 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 200, 50)           70200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200, 50)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 10001     
Total params: 80,201
Trainable params: 80,201
Non-trainable params: 0
_________________________________________________________________


In [14]:
#P.290 訓練時間少一半，準確度卻沒降很多
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test)) 

model_structure = model.to_json() 
with open("lstm_model7.json", "w") as json_file: 
    json_file.write(model_structure)

model.save_weights("lstm_weights7.h5")

Train on 10000 samples, validate on 2500 samples
Epoch 1/2
10000/10000 [==============================] - 99s 10ms/step - loss: 0.5207 - accuracy: 0.7336 - val_loss: 0.4291 - val_accuracy: 0.8144
Epoch 2/2
10000/10000 [==============================] - 98s 10ms/step - loss: 0.3952 - accuracy: 0.8288 - val_loss: 0.4828 - val_accuracy: 0.7664


In [15]:
#P.293 試著做所有字符的建模，會發現有overfitting的問題
#但經過集中培訓的情況下，可對一種特定類型的語言進行建模
dataset = pre_process_data('./aclimdb/train') 
expected = collect_expected(dataset)

In [16]:
#P.293 樣本中平均有多少個token
def avg_len(data): 
    total_len = 0 
    for sample in data: 
        total_len += len(sample[1]) 
    return total_len/len(data)

avg_len(dataset) 

1325.06964

In [17]:
#P.293-294 清除與自然語言無關的字母以及無用的標點字符
def clean_data(data): 
    """轉換為小寫, 用UNK替代不知道的token""" 
    new_data = [] 
    VALID = 'abcdefghijklmnopqrstuvwxyz0123456789"\'?!.,:; ' 
    for sample in data: 
        new_sample = [] 
        for char in sample[1].lower():#取出逐個英文字母 
            if char in VALID: #看是否有在VALID
                new_sample.append(char) #在的話直接加進list
            else: 
                new_sample.append('UNK') #不在用UNK替代
        new_data.append(new_sample) 
    return new_data

listified_data = clean_data(dataset)#取的每個檔案的逐個英文字母的list

In [18]:
#P.294 填充單個token或truncate
def char_pad_trunc(data, maxlen=1500): 
    """ We truncate to maxlen or add in PAD tokens """ 
    new_dataset = [] 
    for sample in data: 
        if len(sample) > maxlen: 
            new_data = sample[:maxlen] 
        elif len(sample) < maxlen: 
            pads = maxlen - len(sample) 
            new_data = sample + ['PAD'] * pads 
        else: 
            new_data = sample 
        new_dataset.append(new_data) 
    return new_dataset

In [19]:
#P.294 為每個字符設indices(索引)
def create_dicts(data): 
    """ Modified from Keras LSTM example""" 
    chars = set() 
    for sample in data: 
        chars.update(set(sample))#用set把重複字母刪除，用update()把每個文檔的set添加進chars裡
    char_indices = dict((c, i) for i, c in enumerate(chars)) 
    indices_char = dict((i, c) for i, c in enumerate(chars)) 
    return char_indices, indices_char

In [20]:
#P.295
import numpy as np

def onehot_encode(dataset, char_indices, maxlen=1500): 
    """ One-hot encode the tokens 
    Args: 
        dataset list of lists of tokens 
        char_indices 
            dictionary of {key=character, value=index to use encoding vector} 
        maxlen int Length of each sample 
    Return: 
        np array of shape (samples, tokens, encoding length) """ 
    X = np.zeros((len(dataset), maxlen, len(char_indices.keys()))) 
    #len(dataset)個maxlen x len(char_indices.keys())的0矩陣
    for i, sentence in enumerate(dataset): 
        for t, char in enumerate(sentence): 
            X[i, t, char_indices[char]] = 1 
    return X

In [21]:
#P.295
dataset = pre_process_data('./aclimdb/train') 
expected = collect_expected(dataset) 
listified_data = clean_data(dataset)

common_length_data = char_pad_trunc(listified_data, maxlen=1500) 
char_indices, indices_char = create_dicts(common_length_data) 
encoded_data = onehot_encode(common_length_data, char_indices, 1500)

In [22]:
#P.295
split_point = int(len(encoded_data)*.4)
split_point_end = int(len(encoded_data)*.5)

x_train = encoded_data[:split_point] 
y_train = expected[:split_point] 
x_test = encoded_data[split_point:split_point_end] 
y_test = expected[split_point:split_point_end]

In [23]:
#P.296
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Embedding, Flatten, LSTM

num_neurons = 40 
maxlen = 1500 
model = Sequential()

model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, len(char_indices.keys())))) 
#return_sequences=False(默認)，只會返回最後一個hidden layer的output
#return_sequences=True，包含全部時間部的hidden layer的output
model.add(Dropout(.2)) 
model.add(Flatten()) 
model.add(Dense(1, activation='sigmoid')) 
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy']) 
model.summary() 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 1500, 40)          14080     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1500, 40)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 60000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 60001     
Total params: 74,081
Trainable params: 74,081
Non-trainable params: 0
_________________________________________________________________


In [24]:
#P.296
batch_size = 32 
epochs = 10 
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test)) 

Train on 10000 samples, validate on 2500 samples
Epoch 1/10
10000/10000 [==============================] - 392s 39ms/step - loss: 0.7147 - accuracy: 0.5278 - val_loss: 0.6960 - val_accuracy: 0.5276
Epoch 2/10
10000/10000 [==============================] - 392s 39ms/step - loss: 0.6077 - accuracy: 0.6869 - val_loss: 0.7129 - val_accuracy: 0.5524
Epoch 3/10
10000/10000 [==============================] - 387s 39ms/step - loss: 0.5184 - accuracy: 0.7615 - val_loss: 0.7654 - val_accuracy: 0.5408
Epoch 4/10
10000/10000 [==============================] - 383s 38ms/step - loss: 0.4312 - accuracy: 0.8201 - val_loss: 0.7888 - val_accuracy: 0.5560
Epoch 5/10
10000/10000 [==============================] - 385s 39ms/step - loss: 0.3573 - accuracy: 0.8523 - val_loss: 0.8631 - val_accuracy: 0.5564
Epoch 6/10
10000/10000 [==============================] - 386s 39ms/step - loss: 0.2894 - accuracy: 0.8891 - val_loss: 0.9554 - val_accuracy: 0.5584
Epoch 7/10
10000/10000 [==============================] -

In [25]:
#P.297
model_structure = model.to_json() 
with open("char_lstm_model3.json", "w") as json_file: 
    json_file.write(model_structure) 
model.save_weights("char_lstm_weights3.h5")

In [26]:
import nltk

nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\i7-870\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


True

In [27]:
#P.301
from nltk.corpus import gutenberg 

gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [28]:
#P.301 資料前處理
text = '' 
for txt in gutenberg.fileids(): 
    if 'shakespeare' in txt: 
        text += gutenberg.raw(txt).lower() #合併所有text
chars = sorted(list(set(text))) #將所有文檔放入set找出使用哪些字符，並改外list排續
char_indices = dict((c, i) for i, c in enumerate(chars)) #為每個字符設indices(索引)
indices_char = dict((i, c) for i, c in enumerate(chars)) 
'corpus length: {} total chars: {}'.format(len(text), len(chars)) 

'corpus length: 375542 total chars: 50'

In [29]:
#P.302 前500字
print(text[:500])

[the tragedie of julius caesar by william shakespeare 1599]


actus primus. scoena prima.

enter flauius, murellus, and certaine commoners ouer the stage.

  flauius. hence: home you idle creatures, get you home:
is this a holiday? what, know you not
(being mechanicall) you ought not walke
vpon a labouring day, without the signe
of your profession? speake, what trade art thou?
  car. why sir, a carpenter

   mur. where is thy leather apron, and thy rule?
what dost thou with thy best apparrell on


In [30]:
#P.302 產生train data
maxlen = 40 
step = 3 
sentences = [] 
next_chars = [] 
for i in range(0, len(text) - maxlen, step):
    #逐步增加三個字符， sentences是從預測字開始的maxlen個字，所以預測字不能超過len(text) - maxlen
    sentences.append(text[i: i + maxlen]) #抓出從預測字開始的40個字
    next_chars.append(text[i + maxlen])#下一個預測的字是往後的第三個字 
print('nb sequences:', len(sentences)) 

nb sequences: 125168


In [31]:
#P.302-303 one-hot
import numpy as np
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) 
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences): 
    for t, char in enumerate(sentence): 
        X[i, t, char_indices[char]] = 1 
        y[i, char_indices[next_chars[i]]] = 1

In [32]:
#P.303
from keras.models import Sequential 
from keras.layers import Dense, Activation 
from keras.layers import LSTM 
from keras.optimizers import RMSprop 

model = Sequential() 
model.add(LSTM(128, input_shape=(maxlen, len(chars)))) #(50+1+128)*128*4=91648
model.add(Dense(len(chars))) #output為50-D #(128+1)*50=6450
model.add(Activation('softmax')) #利用softmax取得50-D的分布概率
optimizer = RMSprop(lr=0.01) #利用權重的最新梯度大小的運行平均值來調整學習率以更新每個權重
model.compile(loss='categorical_crossentropy', optimizer=optimizer) 
model.summary() 

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_4 (Dense)              (None, 50)                6450      
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
Total params: 98,098
Trainable params: 98,098
Non-trainable params: 0
_________________________________________________________________


In [33]:
#P.304 每epochs個時期保存一次模型並保持訓練
epochs = 1
batch_size = 128 

model_structure = model.to_json() 
with open("shakes_lstm_model.json", "w") as json_file: 
    json_file.write(model_structure) 
for i in range(5): 
    model.fit(X, y, batch_size=batch_size, epochs=epochs) 
    #如果停止減少損失，就可以安全地停止過程，並在幾個時期內設置好重量
    model.save_weights("shakes_lstm_weights_{}.h5".format(i+1)) 

Epoch 1/1
125168/125168 [==============================] - 267s 2ms/step - loss: 2.0778
Epoch 1/1
125168/125168 [==============================] - 265s 2ms/step - loss: 1.7076
Epoch 1/1
125168/125168 [==============================] - 264s 2ms/step - loss: 1.5941
Epoch 1/1
125168/125168 [==============================] - 266s 2ms/step - loss: 1.5434
Epoch 1/1
125168/125168 [==============================] - 262s 2ms/step - loss: 1.5990


In [34]:
from keras.models import model_from_json 
with open("shakes_lstm_model.json", "r") as json_file: 
    json_string = json_file.read() 
model = model_from_json(json_string)
model.load_weights("shakes_lstm_weights_{}.h5".format(1))

In [35]:
#P.305 修改多樣性並預測下個字的index #softmax
import random 
def sample(preds, temperature=1.0): 
    preds = np.asarray(preds).astype('float64') 
    preds = np.log(preds) / temperature #總和會變
    #flattening(temperature > 1)或sharpening(temperature < 1)概率分布
    #<1時，更嚴格地重新創建原始文本；>1會產生更多樣化的結果，學習的模式開始被沖走，趨向於胡說八道
    exp_preds = np.exp(preds) #手動softmax
    preds = exp_preds / np.sum(exp_preds) 
    probas = np.random.multinomial(1, preds, 1)#(抽樣次數，抽樣的機率，抽出的數量)
    #抽出的是1，其餘49個是0
    return np.argmax(probas)#輸出擁有最大值的index

In [36]:
#P.306 生成具有不同多樣性的預測文本
import sys 
start_index = random.randint(0, len(text) - maxlen - 1) 
for diversity in [0.2, 0.5, 1.0]: 
    print() 
    print('----- diversity:', diversity) 
    generated = '' 
    sentence = text[start_index: start_index + maxlen]
    generated += sentence 
    print('----- Generating with seed: "' + sentence + '"') 
    sys.stdout.write(generated) #\n時自動換行
    for i in range(400): 
        x = np.zeros((1, maxlen, len(chars))) 
        for t, char in enumerate(sentence): 
            x[0, t, char_indices[char]] = 1. 
        preds = model.predict(x, verbose=0)[0] 
        next_index = sample(preds, diversity) 
        next_char = indices_char[next_index] 
        generated += next_char 
        sentence = sentence[1:] + next_char 
        sys.stdout.write(next_char) 
        sys.stdout.flush() 
    print()


----- diversity: 0.2
----- Generating with seed: " which you deny'd me,
for i can raise no"
 which you deny'd me,
for i can raise not the growne the to so sond the great me to the some the strong the great the strong the strong of the stand and so not the storne and the his brutus, and the the strong the some the strong to the great me to the connertiens and the some the more to the ground the sone the sentle and with a partiens to the ground the strong the will the seete the with the singrons confung, the sent the strones hea

----- diversity: 0.5
----- Generating with seed: " which you deny'd me,
for i can raise no"
 which you deny'd me,
for i can raise not and mad the griue and the mort of the eare,
thered stand where of me strones

   bru.. good with a say the selfe the more in the carsaing and the grocke to withere to the wing it the strones and in my lord

   casc. as in a groble, and wimes the 'tis besties it doones dishelles here

   car. my entrarie,
whose her presse gaue and

In [37]:
#P.308
from keras.models import Sequential 
from keras.layers import GRU 

num_neurons = 40 
model = Sequential() 
model.add(GRU(num_neurons, return_sequences=True, input_shape=X[0].shape))

In [39]:
#P.309
from keras.models import Sequential 
from keras.layers import LSTM 

num_neurons_2 = 40
model = Sequential() 
model.add(LSTM(num_neurons, return_sequences=True, input_shape=X[0].shape)) 
model.add(LSTM(num_neurons_2, return_sequences=True))